In [8]:
from flow.core.params import NetParams
from flow.networks import TrafficLightGridNetwork
from flow.core.params import TrafficLightParams
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, \
    InFlows, SumoCarFollowingParams
from flow.core.params import VehicleParams
import numpy as np

In [31]:
from flow.envs.traffic_light_grid import TrafficLightGridEnv, ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment

In [22]:
# SumoParams, EnvParams
HORIZON=50
sim_params = SumoParams(sim_step=0.2, render=False)
env_params = EnvParams(horizon=HORIZON,
                       additional_params=ADDITIONAL_ENV_PARAMS)

In [39]:
inner_length = 300
long_length = 500
short_length = 300
n = 2 # rows
m = 3 # columns
num_cars_left = 20
num_cars_right = 20
num_cars_top = 20
num_cars_bot = 20
tot_cars = (num_cars_left + num_cars_right) * m \
    + (num_cars_top + num_cars_bot) * n

grid_array = {"short_length": short_length, "inner_length": inner_length,
              "long_length": long_length, "row_num": n, "col_num": m,
              "cars_left": num_cars_left, "cars_right": num_cars_right,
              "cars_top": num_cars_top, "cars_bot": num_cars_bot}

In [40]:
additional_net_params = {"speed_limit": 35, "grid_array": grid_array,
                         "horizontal_lanes": 1, "vertical_lanes": 1,
                         "traffic_lights": True}
network = TrafficLightGridNetwork(name='grid',
                                  vehicles=VehicleParams(),
                                  net_params=NetParams(
                                      additional_params=additional_net_params)
                                 )

In [41]:
env = TrafficLightGridEnv(env_params, sim_params, network)

In [43]:
# exp = Experiment(env)
# _ = exp.run(1, 10000)

In [44]:
net_params = NetParams(additional_params=additional_net_params)
vehicles = VehicleParams()

# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict(
    # name of the experiment
    exp_tag='traffic-rl',
    # name of the flow environment the experiment is running on
    env_name=TrafficLightGridEnv,
    # name of the network class the experiment uses
    network=TrafficLightGridNetwork,
    # simulator that is used by the experiment
    simulator='traci',
    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=sim_params,
    # environment related parameters (see flow.core.params.EnvParams)
    env=env_params,
    # network-related parameters (see flow.core.params.NetParams and
    # the network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,
    # vehicles to be placed in the network at the start of a rollout 
    # (see flow.core.vehicles.Vehicles)
    veh=vehicles,
)

In [36]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

In [38]:
# number of parallel workers
N_CPUS = 1
# number of rollouts per training iteration
N_ROLLOUTS = 1

ray.init(num_cpus=N_CPUS)

2019-11-30 22:18:41,405	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-11-30_22-18-41_405270_28103/logs.
2019-11-30 22:18:41,522	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:22558 to respond...
2019-11-30 22:18:41,644	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:52100 to respond...
2019-11-30 22:18:41,648	INFO services.py:809 -- Starting Redis shard with 3.44 GB max memory.
2019-11-30 22:18:41,680	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-11-30_22-18-41_405270_28103/logs.


ValueError: The default object store size of 5.15 GB will use more than 90% of the available memory on this node (5.43 GB). Please reduce the object store memory size to avoid memory contention with other applications, or shut down the applications using this memory.